## Formulation

In [60]:
#Constants
import math
import numpy as np


Cp=1004 #J/Kg/k
Lv=2.5*math.pow(10,6) #J/kg
eta=0.622
Rd=287 #J/Kg/k
Rv=Rd/eta #J/Kg/k
g=9.8 #m/s2
gamma_d=g/Cp 
rho_a=1.225 #kg/m3

### Estimation of saturation mixing ratio <br>
<img  width=400 src="../formulas/q.png">


In [61]:

def estimation_of_q(T,p):
    c1=Lv/Rv
    e=6.11*np.exp(c1*((1/273)-(1/T)))
    q=(eta*e)/p
    return q

### Estimation of Moist Temperature Lapse rate <br>
<img  width=200 src="../formulas/gamma_m.png">

In [62]:
def estimation_of_gamma_m(q,T):
    numerator=1+((Lv*q)/(Rd*T))
    denom=1+((Lv*Lv*q)/(Cp*Rv*T*T))
    gamma_m=gamma_d*(numerator/denom)
    return gamma_m

### Estimation of Condensation rate <br>
<img  width=400 src="../formulas/Cw.png">

In [63]:
def estimation_of_Cw(T,p):
    q=estimation_of_q(T,p)
    gamma_m=estimation_of_gamma_m(q,T)
    Cw=rho_a*(Cp/Lv)*(gamma_d-gamma_m)
    return Cw

### Estimation of Nd <br>
<img  width=400 src="../formulas/Nd.png">

In [64]:

def estimation_of_Nd(cloud_T,cloud_p,tau,re):
    k=0.8
    f_ad=1
    Cw=estimation_of_Cw(cloud_T,cloud_p)
    Q_ext=2
    rho_w=1000 #kg/m3

    c1=(np.sqrt(5)/(2*math.pi*k))
    numerator=f_ad*Cw*tau
    denom=Q_ext*rho_w*np.power(re,5)

    Nd=c1*np.sqrt(numerator/denom)
    return Nd

# Reading the Dataset and calucation

In [65]:
import xarray as xr

dataDIR="../../Clipped-MYD/outfile.nc"
data = xr.open_dataset(dataDIR)

# cloud_T=data.Cloud_Top_Temperature_Mean.sel(time='2005-03-01T00:00:00.000000000')
# cloud_p=data.Cloud_Top_Pressure_Mean.sel(time='2005-03-01T00:00:00.000000000')
# re=data.Cloud_Effective_Radius_Liquid_Mean.sel(time='2005-03-01T00:00:00.000000000')
# tau=data.Cloud_Optical_Thickness_Combined_Mean.sel(time='2005-03-01T00:00:00.000000000')


cloud_T=data.Cloud_Top_Temperature_Mean
cloud_p=data.Cloud_Top_Pressure_Mean
re=data.Cloud_Effective_Radius_Liquid_Mean
tau=data.Cloud_Optical_Thickness_Combined_Mean

cloud_T=cloud_T[:,:,:].astype(np.double)
cloud_p=cloud_p[:,:,:].astype(np.double)
re=re[:,:,:].astype(np.double)
tau=tau[:,:,:].astype(np.double)

Nd=estimation_of_Nd(cloud_T,cloud_p,tau,re)

Nd.to_netcdf("Nd.nc")
